# Exploration 7 Project - Movielens 영화 추천 실습

[이전 스텝](./Explorationi7(1).ipynb)에서 배운 MF 모델 학습 방법을 토대로, 내가 좋아할만한 영화 추천 시스템을 제작해 보겠습니다.
이번에 활용할 데이터셋은 추천시스템의 MNIST라고 부를만한 **Movielens 데이터**입니다.

- 유저가 영화에 대해 평점을 매긴 데이터가 데이터 크기 별로 있습니다. `MovieLens 1M Dataset` 사용을 권장합니다.
- 별점 데이터는 대표적인 explicit 데이터입니다. 하지만 implicit 데이터로 간주하고 테스트해볼 수 있습니다.
- 별점을 **시청횟수**로 해석해서 생각하겠습니다.
- 또한 유저가 3점 미만으로 준 데이터는 선호하지 않는다고 가정하고 제외하겠습니다.

`MovieLens 1M Dataset` 데이터셋의 경우는 크기가 작아서 아래와 같이 직접 다운 받고 압축해제 하겠습니다.

```bash
1) wget으로 데이터 다운로드
$ wget http://files.grouplens.org/datasets/movielens/ml-1m.zip

2) 다운받은 데이터를 작업디렉토리로 이동
$ mv ml-1m.zip ~/aiffel/recommendata_iu/data

3) gzip으로 압축된 압축을 해제하면 tar 파일이 하나 나옵니다. 
$ unzip ml-1m.zip
```

## 1. 데이터 준비와 전처리


Movielens 데이터는 `rating.dat` 안에 이미 인덱싱까지 완료된 사용자-영화-평점 데이터가 깔끔하게 정리되어 있습니다.

In [34]:
import os
import pandas as pd

rating_file_path = os.path.dirname(os.path.abspath('__file__'))
rating_file_path += r'/data/ml-1m/ratings.dat'
rating_cols = ['user_id', 'movie_id', 'rating', 'timestamp']

ratings = pd.read_csv(rating_file_path, sep='::', names=rating_cols, engine='python')
original_data_size = len(ratings)
ratings.head(10)

,user_id,movie_id,rating,timestamp
0,1,1193,5,978300760
1,1,661,3,978302109
2,1,914,3,978301968
3,1,3408,4,978300275
4,1,2355,5,978824291
5,1,1197,3,978302268
6,1,1287,5,978302039
7,1,2804,5,978300719
8,1,594,4,978302268
9,1,919,4,978301368


In [35]:
# 3점 이상만 남깁니다.
ratings = ratings[ratings['rating'] >= 3]
filtered_data_size = len(ratings)

print(f'orginal_data_size: {original_data_size}, filtered_data_size: {filtered_data_size}')
print(f'Ratio of Remaining Data is {filtered_data_size / original_data_size:.2%}')

orginal_data_size: 1000209, filtered_data_size: 836478
Ratio of Remaining Data is 83.63%


In [36]:
# rating 칼럼의 이름을 count로 바꿉니다.
ratings.rename(columns={'rating' : 'count'}, inplace = True)

In [37]:
# 영화 제목을 보기 위해 메타 데이터를 읽어옵니다.
movie_file_path = os.path.dirname(os.path.abspath('__file__'))
movie_file_path += r'/data/ml-1m/movies.dat'
cols = ['movie_id', 'title', 'genre']

movies = pd.read_csv(movie_file_path, sep='::', names=cols, engine='python')
movies.head(10)

,movie_id,title,genre
0,1,Toy Story (1995),Animation|Children's|Comedy
1,2,Jumanji (1995),Adventure|Children's|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama
4,5,Father of the Bride Part II (1995),Comedy
5,6,Heat (1995),Action|Crime|Thriller
6,7,Sabrina (1995),Comedy|Romance
7,8,Tom and Huck (1995),Adventure|Children's
8,9,Sudden Death (1995),Action
9,10,GoldenEye (1995),Action|Adventure|Thriller


## 2. 분석해봅시다.

- ratings에 있는 유니크한 영화 개수
- rating에 있는 유니크한 사용자 수
- 가장 인기있는 영화 30개 (인기순)


1. ratings에 있는 유니크한 영화 개수

In [9]:
# 유니크한 영화 개수
ratings['movie_id'].nunique()

3628

2. rating에 있는 유니크한 사용자 수

In [10]:
ratings['user_id'].nunique()

6039

3. 가장 인기있는 영화 30개 (인기순으로 출력)

영화 이름을 출력하고 싶으니 movies 데이터 프레임과 ratings 데이터 프레임을 합쳐줍시다.

In [50]:
new_df = pd.merge(ratings, movies, on='movie_id')

In [46]:
new_df.head(30)

,user_id,movie_id,count,timestamp,title,genre
0,1,1193,5,978300760,One Flew Over the Cuckoo's Nest (1975),Drama
1,2,1193,5,978298413,One Flew Over the Cuckoo's Nest (1975),Drama
2,12,1193,4,978220179,One Flew Over the Cuckoo's Nest (1975),Drama
3,15,1193,4,978199279,One Flew Over the Cuckoo's Nest (1975),Drama
4,17,1193,5,978158471,One Flew Over the Cuckoo's Nest (1975),Drama
5,18,1193,4,978156168,One Flew Over the Cuckoo's Nest (1975),Drama
6,19,1193,5,982730936,One Flew Over the Cuckoo's Nest (1975),Drama
7,24,1193,5,978136709,One Flew Over the Cuckoo's Nest (1975),Drama
8,28,1193,3,978125194,One Flew Over the Cuckoo's Nest (1975),Drama
9,33,1193,5,978557765,One Flew Over the Cuckoo's Nest (1975),Drama


In [47]:
new_df.tail(30)

,user_id,movie_id,count,timestamp,title,genre
836448,5039,2576,3,962511591,"Love, etc. (1996)",Drama
836449,5059,1434,4,962484364,"Stranger, The (1994)",Action
836450,5947,1434,4,957190428,"Stranger, The (1994)",Action
836451,5077,1868,3,962417299,"Truce, The (1996)",Drama|War
836452,5105,404,3,962337582,Brother Minister: The Assassination of Malcolm...,Documentary
836453,5185,404,4,963402617,Brother Minister: The Assassination of Malcolm...,Documentary
836454,5532,404,5,959619841,Brother Minister: The Assassination of Malcolm...,Documentary
836455,5543,404,3,960127592,Brother Minister: The Assassination of Malcolm...,Documentary
836456,5153,3463,3,961969400,Last Resort (1994),Comedy
836457,5182,2486,3,961787615,24-hour Woman (1998),Drama


In [22]:
movie_count = new_df.groupby('title')['title'].count()
movie_count.sort_values(ascending=False).head(30)

title
American Beauty (1999)                                   3211
Star Wars: Episode IV - A New Hope (1977)                2910
Star Wars: Episode V - The Empire Strikes Back (1980)    2885
Star Wars: Episode VI - Return of the Jedi (1983)        2716
Saving Private Ryan (1998)                               2561
Terminator 2: Judgment Day (1991)                        2509
Silence of the Lambs, The (1991)                         2498
Raiders of the Lost Ark (1981)                           2473
Back to the Future (1985)                                2460
Matrix, The (1999)                                       2434
Jurassic Park (1993)                                     2413
Sixth Sense, The (1999)                                  2385
Fargo (1996)                                             2371
Braveheart (1995)                                        2314
Men in Black (1997)                                      2297
Schindler's List (1993)                                  2257
Pr

잘 출력되었습니다! American Beauty가 1위라니 놀랍네요. 스타워즈는 4, 5, 6 시리즈가 나란히 2, 3, 4위입니다! 역시 스타워즈!!

## 3. 내가 선호하는 영화를 5가지 골라서 ratings에 추가해줍시다.

내가 좋아하는 영화를 5가지 골라 추가해봅시다. 

In [51]:
# 본인이 좋아하는 영화 데이터로 바꿔서 추가하세요!
my_favorite = [
    'Pulp Fiction (1994)',
    'Toy Story (1995)',
    'Fight Club (1999)',
    'Titanic (1997)',
    'Mission: Impossible (1996)'
]

# 'jiyeon'이라는 user_id가 위 아티스트의 노래를 5회씩 봤다고 가정합시다.
my_rating = pd.DataFrame({'user_id' : ['jiyeon'] * 5, 'title' : my_favorite, 'count' : [5] * 5})

# user_id에 'jiyeon'이라는 데이터가 없으면 위에 임의로 만든 my_favorite 데이터를 추가해줍니다.
if not new_df.isin({'user_id' : ['jiyeon']})['user_id'].any():
    new_df = new_df.append(my_rating)

new_df.tail(10)

,user_id,movie_id,count,timestamp,title,genre
836473,5851,3607.0,5,9.577566e+08,One Little Indian (1973),Comedy|Drama|Western
836474,5854,3026.0,4,9.583469e+08,Slaughterhouse (1987),Horror
836475,5854,690.0,3,9.577443e+08,"Promise, The (Versprechen, Das) (1994)",Romance
836476,5938,2909.0,4,9.572734e+08,"Five Wives, Three Secretaries and Me (1998)",Documentary
836477,5948,1360.0,5,1.016564e+09,Identification of a Woman (Identificazione di ...,Drama
0,jiyeon,NaN,5,NaN,Pulp Fiction (1994),NaN
1,jiyeon,NaN,5,NaN,Toy Story (1995),NaN
2,jiyeon,NaN,5,NaN,Fight Club (1999),NaN
3,jiyeon,NaN,5,NaN,Titanic (1997),NaN
4,jiyeon,NaN,5,NaN,Mission: Impossible (1996),NaN


In [67]:
new_df.tail(10)

,user_id,movie_id,count,timestamp,title,genre
836473,5851,3607.0,5,9.577566e+08,One Little Indian (1973),Comedy|Drama|Western
836474,5854,3026.0,4,9.583469e+08,Slaughterhouse (1987),Horror
836475,5854,690.0,3,9.577443e+08,"Promise, The (Versprechen, Das) (1994)",Romance
836476,5938,2909.0,4,9.572734e+08,"Five Wives, Three Secretaries and Me (1998)",Documentary
836477,5948,1360.0,5,1.016564e+09,Identification of a Woman (Identificazione di ...,Drama
0,jiyeon,NaN,5,NaN,Pulp Fiction (1994),NaN
1,jiyeon,NaN,5,NaN,Toy Story (1995),NaN
2,jiyeon,NaN,5,NaN,Fight Club (1999),NaN
3,jiyeon,NaN,5,NaN,Titanic (1997),NaN
4,jiyeon,NaN,5,NaN,Mission: Impossible (1996),NaN


### 모델에 활용하기 위한 전처리

인덱싱 하는 작업을 해줍시다!

In [102]:
df = pd.merge(new_df, movies, on=['title'])

In [103]:
df.tail(10)

,user_id,movie_id_x,count,timestamp,title,genre_x,movie_id_y,genre_y
836473,5494,3530.0,4,9.598163e+08,Smoking/No Smoking (1993),Comedy,3530,Comedy
836474,5556,2198.0,3,9.594455e+08,Modulations (1998),Documentary,2198,Documentary
836475,5949,2198.0,5,9.588464e+08,Modulations (1998),Documentary,2198,Documentary
836476,5675,2703.0,3,9.760291e+08,Broken Vessels (1998),Drama,2703,Drama
836477,5717,2258.0,4,9.585094e+08,Master Ninja I (1984),Action,2258,Action
836478,5851,3607.0,5,9.577566e+08,One Little Indian (1973),Comedy|Drama|Western,3607,Comedy|Drama|Western
836479,5854,3026.0,4,9.583469e+08,Slaughterhouse (1987),Horror,3026,Horror
836480,5854,690.0,3,9.577443e+08,"Promise, The (Versprechen, Das) (1994)",Romance,690,Romance
836481,5938,2909.0,4,9.572734e+08,"Five Wives, Three Secretaries and Me (1998)",Documentary,2909,Documentary
836482,5948,1360.0,5,1.016564e+09,Identification of a Woman (Identificazione di ...,Drama,1360,Drama


In [104]:
df.rename(columns={"movie_id_x":"movie_id", 'genre_x' : 'genre'}, inplace = True)

In [105]:
del df['movie_id_y'], df['genre_y']
df.tail(10)

,user_id,movie_id,count,timestamp,title,genre
836473,5494,3530.0,4,9.598163e+08,Smoking/No Smoking (1993),Comedy
836474,5556,2198.0,3,9.594455e+08,Modulations (1998),Documentary
836475,5949,2198.0,5,9.588464e+08,Modulations (1998),Documentary
836476,5675,2703.0,3,9.760291e+08,Broken Vessels (1998),Drama
836477,5717,2258.0,4,9.585094e+08,Master Ninja I (1984),Action
836478,5851,3607.0,5,9.577566e+08,One Little Indian (1973),Comedy|Drama|Western
836479,5854,3026.0,4,9.583469e+08,Slaughterhouse (1987),Horror
836480,5854,690.0,3,9.577443e+08,"Promise, The (Versprechen, Das) (1994)",Romance
836481,5938,2909.0,4,9.572734e+08,"Five Wives, Three Secretaries and Me (1998)",Documentary
836482,5948,1360.0,5,1.016564e+09,Identification of a Woman (Identificazione di ...,Drama


In [94]:
# 고유한 유저, 영화를 찾아내는 코드
user_unique = df['user_id'].unique()
movie_unique = df['title'].unique()

# 유저, 영화 indexing 하는 코드.
user_to_idx = {v : k for k, v in enumerate(user_unique)}
movie_to_idx = {v : k for k, v in enumerate(movie_unique)}

In [99]:
# 인덱싱 제대로 되었는지 확인
print(user_to_idx['jiyeon'])
print(movie_to_idx['Modulations (1998)'])

5501
3620


In [106]:
# indexing을 통해 데이터 칼럼 내 값을 바꾸는 코드

temp_user_data = df['user_id'].map(user_to_idx.get).dropna()

if len(temp_user_data) == len(df):
    print('user_id column indexing OK!!')
    df['user_id'] = temp_user_data
else:
    print('user_id column indexing fail')
    
temp_movie_data = df['title'].map(movie_to_idx.get).dropna()
if len(temp_movie_data) == len(df):
    print('movie column indexing ok!!')
    df['movie_id2'] = temp_movie_data
else:
    print('movie column indexing fail')

df.head(10)

user_id column indexing OK!!
movie column indexing ok!!


,user_id,movie_id,count,timestamp,title,genre,movie_id2
0,0,1193.0,5,978300760.0,One Flew Over the Cuckoo's Nest (1975),Drama,0
1,1,1193.0,5,978298413.0,One Flew Over the Cuckoo's Nest (1975),Drama,0
2,2,1193.0,4,978220179.0,One Flew Over the Cuckoo's Nest (1975),Drama,0
3,3,1193.0,4,978199279.0,One Flew Over the Cuckoo's Nest (1975),Drama,0
4,4,1193.0,5,978158471.0,One Flew Over the Cuckoo's Nest (1975),Drama,0
5,5,1193.0,4,978156168.0,One Flew Over the Cuckoo's Nest (1975),Drama,0
6,6,1193.0,5,982730936.0,One Flew Over the Cuckoo's Nest (1975),Drama,0
7,7,1193.0,5,978136709.0,One Flew Over the Cuckoo's Nest (1975),Drama,0
8,8,1193.0,3,978125194.0,One Flew Over the Cuckoo's Nest (1975),Drama,0
9,9,1193.0,5,978557765.0,One Flew Over the Cuckoo's Nest (1975),Drama,0


In [107]:
df.tail(30)

,user_id,movie_id,count,timestamp,title,genre,movie_id2
836453,5539,2576.0,3,9.625116e+08,"Love, etc. (1996)",Drama,3604
836454,5631,1434.0,4,9.624844e+08,"Stranger, The (1994)",Action,3605
836455,1647,1434.0,4,9.571904e+08,"Stranger, The (1994)",Action,3605
836456,1397,1868.0,3,9.624173e+08,"Truce, The (1996)",Drama|War,3606
836457,4233,404.0,3,9.623376e+08,Brother Minister: The Assassination of Malcolm...,Documentary,3607
836458,1429,404.0,4,9.634026e+08,Brother Minister: The Assassination of Malcolm...,Documentary,3607
836459,5494,404.0,5,9.596198e+08,Brother Minister: The Assassination of Malcolm...,Documentary,3607
836460,1527,404.0,3,9.601276e+08,Brother Minister: The Assassination of Malcolm...,Documentary,3607
836461,1417,3463.0,3,9.619694e+08,Last Resort (1994),Comedy,3608
836462,3400,2486.0,3,9.617876e+08,24-hour Woman (1998),Drama,3609


In [108]:
df의 user_id와 movie_id2의 값이 모두 정수 인덱스 값으로 변경되었다.

SyntaxError: invalid syntax (<ipython-input-108-96729af5a3e3>, line 1)

## 4. CSR matrix를 직접 만들어봅시다.


In [112]:
from scipy.sparse import csr_matrix

num_user = df['user_id'].nunique()
num_movie = df['movie_id2'].nunique()

print(type(num_user), num_movie)

<class 'int'> 3628


In [126]:
csr_data = csr_matrix(
    (df['count'], (df.user_id, df.movie_id2)),
    shape = (num_user, num_movie)
)
csr_data

<6040x3628 sparse matrix of type '<class 'numpy.longlong'>'
	with 836483 stored elements in Compressed Sparse Row format>

## 5. als_model = AlternatingLeastSquares 모델을 직접 구성하여 훈련시켜봅시다.

MF 모델을 학습해봅시다!!

In [127]:
from implicit.als import AlternatingLeastSquares
import os
import numpy as np

# implicit 라이브러리에서 권장하고 있는 부분. 내용과는 무관하다.
os.environ['OPENBLAS_NUM_THREADS']='1'
os.environ['KMP_DUPLICATE_LIB_OK']='True'
os.environ['MKL_NUM_THREADS']='1'

In [130]:
# Implicit AlternatingLeastSquares 모델 선언
als_model = AlternatingLeastSquares(
    factors = 100,
    regularization = 0.01,
    use_gpu = False,
    iterations=15,
    dtype = np.float32
)

In [131]:
# als모델은 Input으로 반대가 되기 때문에 transpose를 해줍니다.
csr_data_transpose = csr_data.T
csr_data_transpose

<3628x6040 sparse matrix of type '<class 'numpy.longlong'>'
	with 836483 stored elements in Compressed Sparse Column format>

In [132]:
# 모델 훈련
als_model.fit(csr_data_transpose)

모델 학습이 끝났습니다! 한번 확인해볼까요?

## 6. 내가 선호하는 5가지 영화 중 하나와 그 외의 영화 하나를 골라 훈련된 모델이 예측한 나의 선호도를 파악해보세요.

In [133]:
jiyeon, pulpfiction = user_to_idx['jiyeon'], movie_to_idx['Pulp Fiction (1994)']
jiyeon_vector, pulpfiction_vector = als_model.user_factors[jiyeon], als_model.item_factors[pulpfiction]

print("슝=3")

슝=3


In [134]:
jiyeon_vector

array([ 7.3544765e-01, -1.2952182e-01, -9.2475545e-01, -5.8979917e-01,
        4.3272379e-01, -1.9492398e-01,  1.3727584e-01,  5.5871195e-01,
        1.3626143e+00,  1.1331260e+00, -4.8172775e-01,  1.4267854e-01,
        4.5979914e-01, -4.9263835e-01,  4.9398869e-01,  6.4356583e-01,
        3.4746999e-01, -1.2135000e-02,  1.2730372e-01,  1.3622612e-01,
       -8.4791303e-01, -4.5174056e-01,  2.4118029e-01, -9.1317110e-02,
       -9.2349514e-02,  2.2551565e-01,  4.7653931e-01, -1.8318256e-02,
        1.1910729e-02,  4.2170194e-01, -3.2986960e-01,  3.8366884e-01,
        1.8238328e-01, -8.2027382e-01,  3.5433426e-01,  3.4483227e-01,
       -7.9617184e-01, -2.5594819e-01, -7.2263777e-01, -5.4509188e-03,
       -5.3483838e-01,  1.7820582e-01,  7.1280223e-01,  6.9393533e-01,
        3.2104605e-01,  3.3039844e-01,  2.2147883e-01, -3.2164076e-01,
        5.5554432e-01, -6.9421399e-01,  1.1140509e+00,  3.7626183e-01,
        7.5613183e-01, -2.1931335e-01, -3.6700499e-01, -3.7899789e-01,
      

In [135]:
pulpfiction_vector

array([ 4.91291331e-03,  1.32837743e-02,  7.54903769e-03, -1.58622712e-02,
        2.35024765e-02,  2.83065345e-02, -9.03621968e-03,  5.65531012e-03,
        2.16073859e-02,  2.66770255e-02, -1.81010030e-02,  2.73747407e-02,
        1.77206453e-02,  1.85660797e-03, -2.33299379e-05,  3.34669533e-03,
        1.99564919e-02,  2.59697903e-02,  4.34330804e-03, -1.25133228e-02,
        2.59518251e-02, -1.06668314e-02,  2.12280527e-02, -2.31464636e-02,
       -1.35691995e-02,  4.72869910e-03,  2.82860883e-02,  4.38343883e-02,
       -1.04218081e-03,  1.21417479e-03,  1.12044597e-02,  2.77807303e-02,
       -3.74698150e-03, -1.69029143e-02,  1.66910123e-02,  1.72530282e-02,
       -1.44594107e-02,  1.13975359e-02, -2.12299787e-02,  2.30292100e-02,
        1.45885302e-02,  2.31832564e-02,  1.01664746e-02,  3.27771134e-03,
       -1.48376869e-02,  3.94258415e-03,  3.95540195e-03, -2.64741294e-03,
       -3.39749013e-03, -1.65567324e-02,  3.04400716e-02,  5.39243396e-04,
        3.17119919e-02, -

In [136]:
np.dot(jiyeon_vector, pulpfiction_vector)

0.4057696

헉! 펄프픽션과 나의 선호도를 0.4로 판단했습니다. 

그럼 내가 좋아하는 영화와 비슷한 영화를 추천받아 볼까요?

## 7. 내가 좋아하는 영화와 비슷한 영화를 추천받아 봅시다.

In [138]:
favorite_movie = 'Pulp Fiction (1994)'
movie_id = movie_to_idx[favorite_movie]
similar_movie = als_model.similar_items(movie_id, N=15)
similar_movie

[(222, 0.17301287),
 (269, 0.14327908),
 (51, 0.13792525),
 (233, 0.1278663),
 (289, 0.121095344),
 (157, 0.11825231),
 (121, 0.11187404),
 (248, 0.08941873),
 (224, 0.08905278),
 (220, 0.08309609),
 (23, 0.0767827),
 (278, 0.07677217),
 (472, 0.06741407),
 (59, 0.06625727),
 (3536, 0.062290035)]

Tuple 로 반환하고 있습니다. 영화의 id를 다시 영화의 이름으로 매핑시켜 주겠습니다.

In [139]:
idx_to_movie = {v:k for k, v in movie_to_idx.items()}
[idx_to_movie[i[0]] for i in similar_movie]

['Pulp Fiction (1994)',
 'GoodFellas (1990)',
 'Fargo (1996)',
 'Usual Suspects, The (1995)',
 'Reservoir Dogs (1992)',
 'Shawshank Redemption, The (1994)',
 'Silence of the Lambs, The (1991)',
 'Good Will Hunting (1997)',
 'L.A. Confidential (1997)',
 'Seven (Se7en) (1995)',
 "Schindler's List (1993)",
 'Jackie Brown (1997)',
 'Sling Blade (1996)',
 'Few Good Men, A (1992)',
 'Last Klezmer: Leopold Kozlowski, His Life and Music, The (1995)']

대박! 꽤 잘 맞는 것 같죠?

몇번 더 반복하기 위해 위의 코드를 함수로 만들어봅시다.

In [140]:
def get_similar_movie(movie_name : str):
    movie_id = movie_to_idx[movie_name]
    similar_movie = als_model.similar_items(movie_id)
    similar_movie = [idx_to_movie[i[0]] for i in similar_movie]
    return similar_movie

In [141]:
get_similar_movie('Fight Club (1999)')

['Fight Club (1999)',
 'Sixth Sense, The (1999)',
 'Being John Malkovich (1999)',
 'Eyes Wide Shut (1999)',
 'Man on the Moon (1999)',
 'Messenger: The Story of Joan of Arc, The (1999)',
 'American History X (1998)',
 'Go (1999)',
 'End of Days (1999)',
 'Office Space (1999)']

## 8. 내가 가장 좋아할만한 영화들을 추천받아 봅시다.

한번 그럼 추천받아볼가요?

In [142]:
user = user_to_idx['jiyeon']

movie_recommended = als_model.recommend(user, csr_data, N=20, filter_already_liked_items = True)
movie_recommended

[(50, 0.41141018),
 (121, 0.36800143),
 (99, 0.3461219),
 (67, 0.33422166),
 (248, 0.33301622),
 (38, 0.3161317),
 (157, 0.29101747),
 (233, 0.27612472),
 (51, 0.26300085),
 (224, 0.2623604),
 (220, 0.22233883),
 (4, 0.22132914),
 (746, 0.21977329),
 (474, 0.21593629),
 (269, 0.21527371),
 (482, 0.21460986),
 (48, 0.21111208),
 (126, 0.20499876),
 (138, 0.20305249),
 (384, 0.20236059)]

In [143]:
[idx_to_movie[i[0]] for i in movie_recommended]

['Toy Story 2 (1999)',
 'Silence of the Lambs, The (1991)',
 'American Beauty (1999)',
 'Gladiator (2000)',
 'Good Will Hunting (1997)',
 'Sixth Sense, The (1999)',
 'Shawshank Redemption, The (1994)',
 'Usual Suspects, The (1995)',
 'Fargo (1996)',
 'L.A. Confidential (1997)',
 'Seven (Se7en) (1995)',
 "Bug's Life, A (1998)",
 'Perfect Storm, The (2000)',
 'Chicken Run (2000)',
 'GoodFellas (1990)',
 'Mission: Impossible 2 (2000)',
 'Saving Private Ryan (1998)',
 'Shakespeare in Love (1998)',
 'True Lies (1994)',
 'Jerry Maguire (1996)']

대박 토이스토리 2,, Seven 등등을 추천하고 있네요. 딱 제가 좋아하는 것들! 

그럼 왜 토이스토리2를 추천하는지도 한번 확인해봅시다.

In [144]:
toystory2 = movie_to_idx['Toy Story 2 (1999)']
explain = als_model.explain(user, csr_data, itemid=toystory2)

In [145]:
[(idx_to_movie[i[0]], i[1]) for i in explain[1]]

[('Toy Story (1995)', 0.4371000210110684),
 ('Titanic (1997)', 0.01491419565061095),
 ('Pulp Fiction (1994)', -0.011836925463633127),
 ('Mission: Impossible (1996)', -0.013893784079937018),
 ('Fight Club (1999)', -0.01740603529705953)]

토이스토리1을 선호한다는 점이 가장 크게 기여했습니다. 


## 마무리

몇가지 아쉬운 점이 있지만, 꽤 잘 만들어진 모델을 확인할 수 있습니다.